# Parsing Evaluation with Chroma

In [13]:
from chunking_evaluation import GeneralEvaluation
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

import pandas as pd
import numpy as np

from evaluation.segmentation.chroma.chroma_adapter import ChromaChunker
from segmentation.methods.fixed_sized import FixedSizeChunker
from segmentation.methods.recursive import RecursiveChunker

In [4]:
chunkers = [
    FixedSizeChunker(max_tokens=800, overlap=400),
    RecursiveChunker(max_tokens=800, overlap=400),
    FixedSizeChunker(max_tokens=400, overlap=200),
    RecursiveChunker(max_tokens=400, overlap=200),
    FixedSizeChunker(max_tokens=400, overlap=0),
    RecursiveChunker(max_tokens=400, overlap=0),
    FixedSizeChunker(max_tokens=200, overlap=0),
    RecursiveChunker(max_tokens=200, overlap=0),
]

In [5]:
chunkers = [
    ChromaChunker(c) for c in chunkers
]

In [6]:
# Make sure "OPENAI_API_KEY" is set in a .env file
load_dotenv()
embedding_function = OpenAIEmbeddingFunction(model_name="text-embedding-3-large")

In [14]:
evaluation = GeneralEvaluation()

results = []
pubmed_results = []

for chunker in chunkers:
    c_res = evaluation.run(chunker, embedding_function)
    pubmed_scores = c_res["corpora_scores"]["pubmed"]
    del c_res["corpora_scores"]

    c_row = {}
    results.append(c_res)

    for metric, scores in pubmed_scores.items():
        metric_name = metric.replace("_scores", "")
        c_row[f"{metric_name}_mean"] = np.mean(scores)
        c_row[f"{metric_name}_std"] = np.std(scores)

    pubmed_results.append(c_row)

In [15]:
pd.DataFrame(results)

,iou_mean,iou_std,recall_mean,recall_std,precision_omega_mean,precision_omega_std,precision_mean,precision_std
0,0.013501,0.011155,0.881567,0.310999,0.045663,0.030262,0.013507,0.011160
1,0.016065,0.015239,0.845425,0.357840,0.069267,0.054977,0.016069,0.015241


In [16]:
pd.DataFrame(pubmed_results)

,precision_omega_mean,precision_omega_std,iou_mean,iou_std,recall_mean,recall_std,precision_mean,precision_std
0,0.060083,0.034847,0.017296,0.013712,0.785939,0.385819,0.017313,0.013718
1,0.085830,0.062319,0.021435,0.020784,0.736032,0.432026,0.021447,0.020782
